In [1]:

import os

from tqdm.notebook import tqdm as tqdm_notebook
import keras
import numpy as np
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder

import validation.utils as utils
     


2024-01-28 20:30:33.578245: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-28 20:30:33.598147: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-28 20:30:33.598166: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-28 20:30:33.598682: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-28 20:30:33.602104: I tensorflow/core/platform/cpu_feature_guar

In [2]:
tracks = utils.load('dataset/fma_small/tracks.csv')
# features = utils.load('dataset/fma_small/features.csv')
# echonest = utils.load('dataset/fma_small/echonest.csv')

# np.testing.assert_array_equal(features.index, tracks.index)
# assert echonest.index.isin(tracks.index).all()

tracks.shape

(106574, 52)

In [3]:
subset = tracks.index[tracks['set', 'subset'] <= 'small']
assert subset.isin(tracks.index).all()
tracks = tracks.loc[subset]
tracks.shape

(8000, 52)

In [4]:
train = tracks.index[tracks['set', 'split'] == 'training']
val = tracks.index[tracks['set', 'split'] == 'validation']
test = tracks.index[tracks['set', 'split'] == 'test']
print('{} training examples, {} validation examples, {} testing examples'.format(*map(len, [train, val, test])))

genres = list(LabelEncoder().fit(tracks['track', 'genre_top']).classes_)
#genres = list(tracks['track', 'genre_top'].unique())
print('Top genres ({}): {}'.format(len(genres), genres))
genres = list(MultiLabelBinarizer().fit(tracks['track', 'genres_all']).classes_)
print('All genres ({}): {}'.format(len(genres), genres))
     

6400 training examples, 800 validation examples, 800 testing examples
Top genres (8): ['Electronic', 'Experimental', 'Folk', 'Hip-Hop', 'Instrumental', 'International', 'Pop', 'Rock']
All genres (114): [1, 2, 6, 10, 12, 15, 16, 17, 18, 21, 22, 25, 26, 27, 30, 31, 32, 33, 36, 38, 41, 42, 45, 46, 47, 49, 53, 58, 64, 66, 70, 71, 76, 77, 79, 81, 83, 85, 86, 88, 89, 90, 92, 94, 98, 100, 101, 102, 103, 107, 109, 111, 113, 117, 118, 125, 130, 167, 171, 172, 174, 177, 180, 181, 182, 183, 184, 185, 186, 214, 224, 232, 236, 240, 247, 250, 267, 286, 296, 297, 314, 337, 359, 360, 361, 362, 400, 401, 404, 439, 440, 456, 468, 491, 495, 502, 504, 514, 524, 538, 539, 542, 580, 602, 619, 695, 741, 763, 808, 811, 1032, 1060, 1193, 1235]


In [5]:
def genre(g: str) -> str:
    g = g.lower()
    if g == "rock" or g == "instrumental" or g == "electronic": return "rock"
    return "pop"

In [6]:
print(train.shape)
labels=tracks["track", "genre_top"]
print(labels)
print(labels[train])

(6400,)
track_id
2         Hip-Hop
5         Hip-Hop
10            Pop
140          Folk
141          Folk
           ...   
154308    Hip-Hop
154309    Hip-Hop
154413        Pop
154414        Pop
155066    Hip-Hop
Name: (track, genre_top), Length: 8000, dtype: category
Categories (16, object): ['Blues', 'Classical', 'Country', 'Easy Listening', ..., 'Pop', 'Rock', 'Soul-RnB', 'Spoken']
track_id
2         Hip-Hop
5         Hip-Hop
10            Pop
140          Folk
141          Folk
           ...   
153955        Pop
153956        Pop
154413        Pop
154414        Pop
155066    Hip-Hop
Name: (track, genre_top), Length: 6400, dtype: category
Categories (16, object): ['Blues', 'Classical', 'Country', 'Easy Listening', ..., 'Pop', 'Rock', 'Soul-RnB', 'Spoken']


In [7]:
import librosa
from NeuralNetwork.CRNN import CRNN_MEL, CRNN_MFCC
from NeuralNetwork.CNN import CNN_CHR
from NeuralNetwork.combine_model import CombineModel
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [8]:
#new
model_mel = CRNN_MEL((1293,128,1))
model_chr = CNN_CHR((1293,12,1))
model_mfcc = CRNN_MFCC((1293, 20, 1))

# model_mel = CRNN_MEL((128,293,1))
# model_chr = CNN_CHR((12,293,1))
# model_mfcc = CRNN_MFCC(( 20,1293, 1))
combine_model = CombineModel(30)

2024-01-28 20:30:38.567827: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-28 20:30:38.591516: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-28 20:30:38.591634: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 1293, 128, 32)     320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 646, 64, 32)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 646, 64, 64)       18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 323, 32, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 323, 32, 128)      73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 161, 16, 128)      0

/home/lumiani/miniconda3/envs/cs182/lib/python3.11/site-packages/keras/src/preprocessing/image.py:1451: UserWarning: This ImageDataGenerator specifies `zca_whitening` which overrides setting of`featurewise_std_normalization`.
  warnings.warn(


None
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 1293, 20, 32)      320       
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 646, 10, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_9 (Conv2D)           (None, 646, 10, 64)       18496     
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 323, 5, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_10 (Conv2D)          (None, 323, 5, 128)       73856     
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 161, 2, 128) 

In [9]:
model_mel.load_model("./model/model_mel.h5")
model_chr.load_model("./model/model_chr.h5")
model_mfcc.load_model("./model/model_mfcc.h5")
combine_model.model = keras.models.load_model("./model/combine.h5")

In [10]:
acc_cnt = 0
all_cnt = 0
for i in tqdm_notebook(test):
    if labels[i] in ['Experimental', 'Instrumental', 'International']: continue
    filename = f"{i:06d}.mp3"
    file_path = os.path.join("/home/lumiani/Projects/IML/CS182-FinalProject/dataset/fma_small", filename[:3], filename)
    if not os.path.exists(file_path): continue
    y, sr = librosa.load(file_path)
    spectrogram = librosa.feature.melspectrogram(y=y, sr=sr)

    mel_pic = librosa.power_to_db(spectrogram, ref=np.max)
    print(mel_pic.shape)
    if mel_pic.shape[1] < 1293:
        gap = 1293 - mel_pic.shape[1]
        left = gap // 2
        right = gap - left
        mel_pic = np.hstack((np.hstack((np.zeros((128, left)), mel_pic)), np.zeros((128, right))))


    harmonic = librosa.effects.harmonic(y)  ##
    harmonic_features = librosa.feature.chroma_cqt(y=harmonic, sr=sr)  ##

    if harmonic_features.shape[1] < 1293:
        gap = 1293 - harmonic_features.shape[1]
        left = gap // 2
        right = gap - left
        harmonic_features = np.hstack((np.hstack((np.zeros((12, left)), harmonic_features)), np.zeros((12, right))))


    mfccs = librosa.feature.mfcc(y=y, sr=sr)

    if mfccs.shape[1] < 1293:
        gap = 1293 - mfccs.shape[1]
        left = gap // 2
        right = gap - left
        mfccs = np.hstack((np.hstack((np.zeros((20, left)), mfccs)), np.zeros((20, right))))


    duration = len(y) / sr

    # Assuming model1, model2, model3 are already defined with different input shapes but the same output shape
    try:
        _, pre_mel = model_mel.predict(mel_pic, duration)
        _, pre_chr = model_chr.predict(harmonic_features, duration)
        _, pre_mfcc = model_mfcc.predict(mfccs, duration)

        pre_mel=pre_mel.flatten()
        pre_chr=pre_chr.flatten()
        pre_mfcc=pre_mfcc.flatten()

        merge_array=[]
        for arr in [pre_mel, pre_chr, pre_mfcc]:
            merge_array.extend(arr)
        merge_array=np.expand_dims(merge_array, axis=0)
        pre = combine_model.model.predict(merge_array)
        class_idx = np.argmax(pre, axis=1)[0]
        acc_cnt += (["pop", "classical", "pop", "rock", "pop", "jazz", "rock", "pop", "pop", "rock"][class_idx] == genre(labels[i]))
        all_cnt += 1
    except Exception as e:
        print(e)


  0%|          | 0/800 [00:00<?, ?it/s]

(128, 1293)


2024-01-28 20:30:47.468663: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907


1/1 [==============================] - 0s 479ms/step
in user code:

    File "/home/lumiani/miniconda3/envs/cs182/lib/python3.11/site-packages/keras/src/engine/training.py", line 2440, in predict_function  *
        return step_function(self, iterator)
    File "/home/lumiani/miniconda3/envs/cs182/lib/python3.11/site-packages/keras/src/engine/training.py", line 2425, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/lumiani/miniconda3/envs/cs182/lib/python3.11/site-packages/keras/src/engine/training.py", line 2413, in run_step  **
        outputs = model.predict_step(data)
    File "/home/lumiani/miniconda3/envs/cs182/lib/python3.11/site-packages/keras/src/engine/training.py", line 2381, in predict_step
        return self(x, training=False)
    File "/home/lumiani/miniconda3/envs/cs182/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from No

KeyboardInterrupt: 

: 

In [ ]:
print("test accuracy: {}".format(acc_cnt / all_cnt))

ZeroDivisionError: division by zero